In [1]:
import pandas as pd
import numpy as np
import random
import baselines as b
from sklearn.metrics import accuracy_score,f1_score, recall_score, precision_score, average_precision_score, balanced_accuracy_score

In [2]:
path='./data/SAT_Predictions1/'
#path='./data/MRA_Files/'
#path='./data/SAT_findings/'


In [ ]:
sample=[1,2,3,4,5]

bleu1,bleu2,bleu3,bleu4,M, R, C=[],[],[],[],[],[],[]

accuracy,precision,recall_mac,recall_mic=[],[],[],[]

classes=['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Lesion', 'Lung Opacity', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices']

for i in range(0,1000):
    
    print('sample:', i)
#     gt=pd.read_csv(path+'gts_labelled'+str(i)+'.csv')
#     test=pd.read_csv(path+'res_labelled'+str(i)+'.csv')

    #gt=pd.read_csv(path+'ref_labelled'+str(i)+'.csv')
    
    lst=range(240)


    indx=random.choices(lst,k=240)
    
    #test=pd.read_csv(path+'score_labelled'+str(1)+'.csv')
    gt=pd.read_csv(path+'score_ref_labelled'+str(1)+'.csv')
    
#     #print(gt.shape)
    
#     test=pd.read_csv(path+'score_labelled'+str(i)+'.csv')
    
#     print(test.shape)
    
    baselines=pd.read_csv('./data/baselines_latest.csv')
    
    #print(baselines.iloc[:,0])
    test=baselines.iloc[1,:]

    test=pd.DataFrame(test).T

    test=pd.concat([test]*gt.shape[0], ignore_index=True)

    gt=gt.iloc[indx,:]
    
    

    test=test.iloc[indx,:]
    
    gt=gt.fillna(0.0)


    gt=gt.replace(-1,0)

    test=test.fillna(0.0)


    test=test.replace(-1,0)
    
    data=gt.iloc[:,1:]
    
    

    test_data=test.iloc[indx,1:]

    #test_data=test_data.sample(frac=1)
#     n=test_data.shape[0]
#     r=pd.DataFrame(data.iloc[report_id,:]).T

    pred = test_data#r * 1.0
    
    
    #### accuracy=======================================

    accuracy.append(accuracy_score(data.to_numpy().flatten(),test_data.to_numpy().flatten()))

    print(accuracy)
    
  
   
    
#     print(is_normal)
    #test_shuffles=test.sample(frac=1)
    
    print(gt.shape,test.shape)
    bleus,meteor, rouge, ciders=b.all_scores(gt.iloc[:,0],test.iloc[:,0])
#     #bleus,meteor, rouge, ciders=b.all_scores(np.array(gt.iloc[:,0])[is_normal], test.iloc[2,0])
    
    bleu1.append(bleus[0])
    bleu2.append(bleus[1])
    bleu3.append(bleus[2])
    bleu4.append(bleus[3])
    M.append(meteor)
    R.append(rouge)
    C.append(ciders)
    
data=pd.DataFrame([])

data['b1']=bleu1
data['b2']=bleu2
data['b3']=bleu3
data['b4']=bleu4
data['M']=M
data['R']=R
data['C']=C
data['accuracy']=accuracy

print(np.mean(accuracy))
data.to_csv('samples_Base2.csv')

sample: 0
[0.7485119047619048]
(240, 15) (240, 15)
{'testlen': 9120, 'reflen': 9438, 'guess': [9120, 8880, 8640, 8400], 'correct': [4793, 2089, 692, 171]}
ratio: 0.966306420851773
sample: 1
[0.7485119047619048, 0.7398809523809524]
(240, 15) (240, 15)
{'testlen': 9120, 'reflen': 9557, 'guess': [9120, 8880, 8640, 8400], 'correct': [4727, 2051, 682, 174]}
ratio: 0.9542743538766396
sample: 2
[0.7485119047619048, 0.7398809523809524, 0.74375]
(240, 15) (240, 15)
{'testlen': 9120, 'reflen': 9183, 'guess': [9120, 8880, 8640, 8400], 'correct': [4656, 2009, 661, 176]}
ratio: 0.9931394968963309
sample: 3
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191]
(240, 15) (240, 15)
{'testlen': 9120, 'reflen': 9326, 'guess': [9120, 8880, 8640, 8400], 'correct': [4687, 2013, 653, 167]}
ratio: 0.9779112159552886
sample: 4
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762]
(240, 15) (240, 15)
{'testlen': 9120, 'reflen': 8837, 'guess': [9120, 8880, 864

sample: 22
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334]
(240, 15) (240, 15)
{'testlen': 9120, 'reflen': 9163, 'guess': [9120, 8880, 8640, 8400], 'correct': [4601, 1980, 647, 174]}
ratio: 0.9953072137945
sample: 23
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.747916666

sample: 34
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334, 0.7470238095238095, 0.7410714285714286, 0.7473214285714286, 0.7470238095238095, 0.7464285714285714, 0.7386904761904762, 0.736904761904762, 0.7482142857142857, 0.750297619047619, 0.7425595238095238, 0.749702380952381, 0.7523809523809524]
(240, 15) (240, 15)
{'testlen': 9120, 'reflen': 9053, 'guess': [9120, 8880, 8640, 8400], 'correct': [4692, 2058, 678, 181]}
ratio: 1.0074008615927308
sample: 35
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.747023809

sample: 43
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334, 0.7470238095238095, 0.7410714285714286, 0.7473214285714286, 0.7470238095238095, 0.7464285714285714, 0.7386904761904762, 0.736904761904762, 0.7482142857142857, 0.750297619047619, 0.7425595238095238, 0.749702380952381, 0.7523809523809524, 0.7526785714285714, 0.7455357142857143, 0.7485119047619048, 0.7461309523809524, 0.7419642857142857, 0.7422619047619048, 0.7375, 0.7511904761904762, 0.7455357142857143]
(240, 15) (240, 15)
{'testlen': 9120, 'reflen': 9176, 'guess': [9120, 8880, 8640, 8400], 'correct': [4731, 2018, 640, 164]}
ratio: 0.9

sample: 51
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334, 0.7470238095238095, 0.7410714285714286, 0.7473214285714286, 0.7470238095238095, 0.7464285714285714, 0.7386904761904762, 0.736904761904762, 0.7482142857142857, 0.750297619047619, 0.7425595238095238, 0.749702380952381, 0.7523809523809524, 0.7526785714285714, 0.7455357142857143, 0.7485119047619048, 0.7461309523809524, 0.7419642857142857, 0.7422619047619048, 0.7375, 0.7511904761904762, 0.7455357142857143, 0.7461309523809524, 0.7464285714285714, 0.7440476190476191, 0.7526785714285714, 0.7538690476190476, 0.7535714285714286, 0.747023809523

...

sample: 565
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334, 0.7470238095238095, 0.7410714285714286, 0.7473214285714286, 0.7470238095238095, 0.7464285714285714, 0.7386904761904762, 0.736904761904762, 0.7482142857142857, 0.750297619047619, 0.7425595238095238, 0.749702380952381, 0.7523809523809524, 0.7526785714285714, 0.7455357142857143, 0.7485119047619048, 0.7461309523809524, 0.7419642857142857, 0.7422619047619048, 0.7375, 0.7511904761904762, 0.7455357142857143, 0.7461309523809524, 0.7464285714285714, 0.7440476190476191, 0.7526785714285714, 0.7538690476190476, 0.7535714285714286, 0.74702380952

sample: 566
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334, 0.7470238095238095, 0.7410714285714286, 0.7473214285714286, 0.7470238095238095, 0.7464285714285714, 0.7386904761904762, 0.736904761904762, 0.7482142857142857, 0.750297619047619, 0.7425595238095238, 0.749702380952381, 0.7523809523809524, 0.7526785714285714, 0.7455357142857143, 0.7485119047619048, 0.7461309523809524, 0.7419642857142857, 0.7422619047619048, 0.7375, 0.7511904761904762, 0.7455357142857143, 0.7461309523809524, 0.7464285714285714, 0.7440476190476191, 0.7526785714285714, 0.7538690476190476, 0.7535714285714286, 0.74702380952

sample: 567
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334, 0.7470238095238095, 0.7410714285714286, 0.7473214285714286, 0.7470238095238095, 0.7464285714285714, 0.7386904761904762, 0.736904761904762, 0.7482142857142857, 0.750297619047619, 0.7425595238095238, 0.749702380952381, 0.7523809523809524, 0.7526785714285714, 0.7455357142857143, 0.7485119047619048, 0.7461309523809524, 0.7419642857142857, 0.7422619047619048, 0.7375, 0.7511904761904762, 0.7455357142857143, 0.7461309523809524, 0.7464285714285714, 0.7440476190476191, 0.7526785714285714, 0.7538690476190476, 0.7535714285714286, 0.74702380952

sample: 568
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334, 0.7470238095238095, 0.7410714285714286, 0.7473214285714286, 0.7470238095238095, 0.7464285714285714, 0.7386904761904762, 0.736904761904762, 0.7482142857142857, 0.750297619047619, 0.7425595238095238, 0.749702380952381, 0.7523809523809524, 0.7526785714285714, 0.7455357142857143, 0.7485119047619048, 0.7461309523809524, 0.7419642857142857, 0.7422619047619048, 0.7375, 0.7511904761904762, 0.7455357142857143, 0.7461309523809524, 0.7464285714285714, 0.7440476190476191, 0.7526785714285714, 0.7538690476190476, 0.7535714285714286, 0.74702380952

sample: 569
[0.7485119047619048, 0.7398809523809524, 0.74375, 0.7440476190476191, 0.7449404761904762, 0.7425595238095238, 0.7511904761904762, 0.7470238095238095, 0.7431547619047619, 0.7461309523809524, 0.7479166666666667, 0.7407738095238096, 0.7452380952380953, 0.7410714285714286, 0.7443452380952381, 0.7505952380952381, 0.7547619047619047, 0.7392857142857143, 0.7404761904761905, 0.7479166666666667, 0.7419642857142857, 0.7485119047619048, 0.7395833333333334, 0.7470238095238095, 0.7410714285714286, 0.7473214285714286, 0.7470238095238095, 0.7464285714285714, 0.7386904761904762, 0.736904761904762, 0.7482142857142857, 0.750297619047619, 0.7425595238095238, 0.749702380952381, 0.7523809523809524, 0.7526785714285714, 0.7455357142857143, 0.7485119047619048, 0.7461309523809524, 0.7419642857142857, 0.7422619047619048, 0.7375, 0.7511904761904762, 0.7455357142857143, 0.7461309523809524, 0.7464285714285714, 0.7440476190476191, 0.7526785714285714, 0.7538690476190476, 0.7535714285714286, 0.74702380952